<a href="https://colab.research.google.com/github/tomknightatl/USCCB/blob/main/Find_Parish_Directory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# Parish Scraper Jupyter Notebook

import requests
from bs4 import BeautifulSoup
import sqlite3
import re

In [11]:
# Cell 1: Import required libraries and fetch the main page
print("Cell 1: Importing libraries and fetching main page")
url = "https://archatl.com/"
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

Cell 1: Importing libraries and fetching main page


In [12]:
# Cell 2: Find the URL for the parish listing page
print("\nCell 2: Finding the URL for the parish listing page")
parish_link = soup.find('a', text=re.compile('Find a Parish'))
if parish_link:
    parish_url = parish_link['href']
    print(f"Parish listing URL: {parish_url}")
else:
    print("Parish listing URL not found")
    parish_url = None


Cell 2: Finding the URL for the parish listing page
Parish listing URL: https://archatl.com/parishes/find-a-parish/


<ipython-input-12-ca11d5458652>:3: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  parish_link = soup.find('a', text=re.compile('Find a Parish'))


In [13]:
# Cell 3: Fetch and parse the parish listing page
print("\nCell 3: Fetching and parsing the parish listing page")
if parish_url:
    parish_response = requests.get(parish_url)
    parish_soup = BeautifulSoup(parish_response.content, 'html.parser')
    print("Parish listing page fetched successfully")
else:
    print("Cannot fetch parish listing page")
    parish_soup = None


Cell 3: Fetching and parsing the parish listing page
Parish listing page fetched successfully


In [15]:
# Cell 4: Extract parish information and store in SQLite database
print("\nCell 4: Extracting parish information and storing in SQLite database")
conn = sqlite3.connect('parishes.db')
cursor = conn.cursor()

# Create table
cursor.execute('''CREATE TABLE IF NOT EXISTS parishes
                  (name TEXT, address TEXT, url TEXT)''')

if parish_soup:
    # Try different selectors to find parish information
    parishes = parish_soup.find_all('div', class_='parish-item')
    if not parishes:
        parishes = parish_soup.find_all('div', class_='parish')
    if not parishes:
        parishes = parish_soup.find_all('div', class_=re.compile('parish'))

    print(f"Found {len(parishes)} potential parish elements")

    for parish in parishes:
        name = parish.find('h2')
        if name:
            name = name.text.strip()
        else:
            name = "Name not found"

        address = parish.find('p', class_=re.compile('address'))
        if address:
            address = address.text.strip()
        else:
            address = "Address not found"

        url = parish.find('a', href=re.compile('http'))
        if url:
            url = url['href']
        else:
            url = "URL not found"

        cursor.execute("INSERT INTO parishes VALUES (?, ?, ?)", (name, address, url))

    conn.commit()
    print(f"Inserted {len(parishes)} parishes into the database")
else:
    print("No parish information to extract")


Cell 4: Extracting parish information and storing in SQLite database
Found 0 potential parish elements
Inserted 0 parishes into the database


In [6]:
# Cell 5: Verify the data in the SQLite database
print("\nCell 5: Verifying the data in the SQLite database")
cursor.execute("SELECT * FROM parishes LIMIT 5")
rows = cursor.fetchall()
for row in rows:
    print(row)

conn.close()
print("\nDatabase connection closed")


First 5 links:
https://lifeofthechurch.com/donate/
https://archatl.com/parishes/find-a-parish/
https://archatl.com/schools/find-a-school/
https://archatl.com/about/events-calendar/
https://archatl.com/offices/human-resources/job-openings/
